In [ ]:
#Imports
from pathlib import Path
from app.utils.monitoring.data_extractor_for_analysis import DataProvider

from evidently import Report
from evidently.presets import DataDriftPreset

In [ ]:
# Variables d'environnement

# Chemin à charger: on va cherche les artefacts dans le root, datas/raw
# __file__ : chemin du fichier en cours d'exe (model.py)
# resolve(): convertie en chemin absolu + parent: donne le dossier du parent
DATAS_PATH = Path(__file__).resolve().parent.parent / "datas/raw"
SAVE_PATH = Path(__file__).resolve().parent.parent / "datas/results"
# DB_URL = "postgresql://postgres:[PASSWORD]@[HOST]:5432/postgres"
MONITORED_VAR = [
    "EXT_SOURCE_COUNT",
    "DAYS_EMPLOYED",
    "DAYS_BIRTH",
    "AMT_ANNUITY",
    "PAYMENT_RATE",
    "PREV_DAYS_LAST_DUE_1ST_VERSION_MAX",
    "BUREAU_AMT_CREDIT_SUM_DEBT_MEAN",
    "OWN_CAR_AGE",
    "NAME_FAMILY_STATUS",
]

In [ ]:
# Instanciation
provider = DataProvider()

In [ ]:
# Chargement de la données
# ============= Trop lourd en local mais réalisé avec google colab ===============
# df_ref = provider.load_reference_data(DATAS_PATH/"Xy_hist.parquet")
# # df_current = provider.fetch_datas_from_db(len(df_ref))
# df_current = provider.load_potential_data_drift(DATAS_PATH/"10000"/"predictions_rows.csv")
# ============= cf dossier results/figures/data_drift ===============
# Cas allégé avec hist de 10% et drift de 10000 en parquet
df_ref = provider.load_reference_data(
    DATAS_PATH/"Xy_hist_sample10percents.parquet"
)
# df_current = provider.fetch_datas_from_db(len(df_ref))
df_current = provider.load_potential_data_drift(
    DATAS_PATH/"Xy_drifted_strongAugmentations_sample10000_noTarget.parquet"
)

# Alignement (important pour éviter les erreurs de colonnes)
df_ref, df_current = provider.align_datasets(df_ref, df_current)

In [ ]:
# Configuration du Rapport
report = Report(metrics=[
    DataDriftPreset(columns=MONITORED_VAR)
])

# Exécution
eval1 = report.run(reference_data=df_ref, current_data=df_current)

In [ ]:
# Sauvegarde et visu
eval1.save_html(str(SAVE_PATH/"data_drift/eval1.html"))
eval1

**Seuil**:

Le Dataset Drift threshold n'est pas le seuil de classification métier (0.511 pour solvable/insolvable), mais un seuil statistique de décision.

Par défaut (0.5) : Evidently considère qu'il y a un drift global du dataset si plus de 50% des colonnes analysées présentent un drift individuel.

Calcul interne : Si analyses 10 colonnes (MONITORED_VAR), et que 4 d'entre elles dérivent, le ratio est de 0.4. Comme $0.4 < 0.5$, le rapport affiche : "Dataset Drift is NOT detected".

**Tableau**:

- Column: Le nom des features, si on clique sur la flèche on a un détail de la donnée en terme de drifting et de distribution
- Type: le type de la feature (num, cat...)
- Reference/Current Distribution: une miniature dynamique des ditributions des deux jeux de données
- Data Drift: L'avis d'evidently sur le jeu de donnée courant (drift detected ou pas)
- Stat Test: Le test employé par evidently pour juger (dans notre cas Wasserstein distance (normed) pour les numériques et Jensen-Shannon distance pour le catégoriel). Remarque: Evidently bascule sur Wasserstein quand le volume de donnée est trop importante car les tests en P-value y seraient trop sensible (de même pour Jensen-Shannon au lieu de Chi2)
- Drift Score : Score de drift lié au test. 
    - Pour Wassertein normé: 
        * $0.0$ à $0.1$ : Drift négligeable (les distributions sont quasi superposées).
        * $0.1$ à $0.25$ : Drift faible (mouvement mineur).
        * $> 0.33$ : Drift détecté par défaut dans Evidently.
    - Pour Jensen-Shanon:
        * $0$ : Les distributions sont identiques.
        * $1$ : Les distributions sont totalement disjointes.